### Phần A: Xử lý Dữ liệu

##### 1. Import thư viện và Cấu hình

In [5]:
%pip install torch numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from collections import Counter
import numpy as np
import random

# Thiết lập seed để kết quả tái lập được
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


##### 2. Tokenization & Xây dựng Vocabulary

In [25]:
class Vocabulary:
    def __init__(self, freq_threshold=2):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer(text):
        # Tách từ đơn giản bằng khoảng trắng và đưa về chữ thường
        # Trong thực tế nên dùng thư viện như underthesea (cho tiếng Việt) hoặc spacy
        return text.lower().strip().split()

    def build_vocabulary(self, sentence_list):
        frequencies = Counter()
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer(sentence):
                frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer(text)
        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

# Hàm đọc file
def read_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return f.readlines()

# Tải dữ liệu (Giả sử file nằm ở thư mục hiện tại)
try:
    src_lines = read_file('train.vi.txt') # Tiếng Việt là nguồn
    trg_lines = read_file('train.en.txt') # Tiếng Anh là đích
    print(f"Số lượng câu huấn luyện: {len(src_lines)}")
except FileNotFoundError:
    print("Vui lòng đảm bảo các file 'train.vi.txt' và 'train.en.txt' đã được tải lên.")

# Xây dựng Vocab
src_vocab = Vocabulary(freq_threshold=2)
src_vocab.build_vocabulary(src_lines)

trg_vocab = Vocabulary(freq_threshold=2)
trg_vocab.build_vocabulary(trg_lines)

print(f"Kích thước từ điển nguồn (VI): {len(src_vocab)}")
print(f"Kích thước từ điển đích (EN): {len(trg_vocab)}")

Số lượng câu huấn luyện: 500000
Kích thước từ điển nguồn (VI): 119287
Kích thước từ điển đích (EN): 183367


##### 3. Tạo Dataset và DataLoader

In [8]:
class TranslationDataset(data.Dataset):
    def __init__(self, src_lines, trg_lines, src_vocab, trg_vocab):
        self.src_lines = src_lines
        self.trg_lines = trg_lines
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab

    def __len__(self):
        return len(self.src_lines)

    def __getitem__(self, index):
        src_text = self.src_lines[index]
        trg_text = self.trg_lines[index]

        src_indices = [self.src_vocab.stoi["<SOS>"]] + \
                      self.src_vocab.numericalize(src_text) + \
                      [self.src_vocab.stoi["<EOS>"]]
        
        trg_indices = [self.trg_vocab.stoi["<SOS>"]] + \
                      self.trg_vocab.numericalize(trg_text) + \
                      [self.trg_vocab.stoi["<EOS>"]]

        return torch.tensor(src_indices), torch.tensor(trg_indices)

class Collate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        src = [item[0] for item in batch]
        trg = [item[1] for item in batch]
        
        # Padding
        src = nn.utils.rnn.pad_sequence(src, batch_first=True, padding_value=self.pad_idx)
        trg = nn.utils.rnn.pad_sequence(trg, batch_first=True, padding_value=self.pad_idx)
        
        return src, trg

# Thiết lập DataLoader
BATCH_SIZE = 32
PAD_IDX = src_vocab.stoi["<PAD>"]

dataset = TranslationDataset(src_lines, trg_lines, src_vocab, trg_vocab)
dataloader = data.DataLoader(
    dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=Collate(pad_idx=PAD_IDX)
)

### Phần B: Xây dựng Kiến trúc Transformer From Scratch

##### 1. Embeddings & Positional Encoding

In [9]:
class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        # Scaling embedding theo căn bậc 2 của d_model (theo paper)
        return self.lut(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Tạo ma trận encoding
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        # Công thức Sin/Cos
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        pe = pe.unsqueeze(0) # Thêm dimension batch: [1, max_len, d_model]
        self.register_buffer('pe', pe) # Không train tham số này

    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        # Cộng positional encoding vào embedding
        x = x + self.pe[:, :x.size(1)].requires_grad_(False)
        return x

##### 2. Multi-Head Attention

In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        assert d_model % n_head == 0
        
        self.d_k = d_model // n_head
        self.n_head = n_head
        self.d_model = d_model
        
        # Các lớp Linear để chiếu Q, K, V
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.fc = nn.Linear(d_model, d_model)
        
    def attention(self, query, key, value, mask=None):
        # query, key, value: [batch_size, n_head, seq_len, d_k]
        
        # 1. Matmul Q và K
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # 2. Masking (nếu có)
        if mask is not None:
            # Thay thế vị trí mask = 0 bằng số rất nhỏ (-1e9) để softmax về 0
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # 3. Softmax
        p_attn = torch.softmax(scores, dim=-1)
        
        # 4. Matmul với V
        return torch.matmul(p_attn, value), p_attn

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        
        # 1. Linear projections & Split heads
        # view: [batch_size, seq_len, n_head, d_k] -> transpose: [batch_size, n_head, seq_len, d_k]
        q = self.w_q(query).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
        k = self.w_k(key).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
        v = self.w_v(value).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
        
        # 2. Scaled Dot-Product Attention
        x, self.attn = self.attention(q, k, v, mask=mask)
        
        # 3. Concat heads
        # transpose: [batch_size, seq_len, n_head, d_k] -> view: [batch_size, seq_len, d_model]
        x = x.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # 4. Final Linear
        return self.fc(x)

##### 3. Position-wise Feed-Forward & SublayerConnection

In [11]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        # Linear -> ReLU -> Dropout -> Linear
        return self.w_2(self.dropout(self.relu(self.w_1(x))))

class SublayerConnection(nn.Module):
    """
    Thực hiện: x + Dropout(Sublayer(Norm(x))) (Pre-Norm) hoặc x + Norm(Dropout(Sublayer(x))) (Post-Norm)
    Ở đây ta cài đặt Post-Norm tiêu chuẩn: Norm(x + Sublayer(x))
    """
    def __init__(self, d_model, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        return self.norm(x + self.dropout(sublayer(x)))

##### 4. Encoder & Decoder Layer

In [12]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_head)
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(2)])
        self.d_model = d_model

    def forward(self, x, mask):
        # 1. Multi-Head Self Attention
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        # 2. Feed Forward
        x = self.sublayer[1](x, self.feed_forward)
        return x

class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, n_head)
        self.src_attn = MultiHeadAttention(d_model, n_head) # Cross Attention
        self.feed_forward = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.sublayer = nn.ModuleList([SublayerConnection(d_model, dropout) for _ in range(3)])
        self.d_model = d_model

    def forward(self, x, memory, src_mask, trg_mask):
        # memory là output từ Encoder
        
        # 1. Masked Self Attention (Decoder chỉ nhìn thấy từ quá khứ)
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, trg_mask))
        
        # 2. Cross Attention (Query=Decoder, Key/Value=Encoder)
        x = self.sublayer[1](x, lambda x: self.src_attn(x, memory, memory, src_mask))
        
        # 3. Feed Forward
        x = self.sublayer[2](x, self.feed_forward)
        return x

##### 5. Ghép nối thành Transformer hoàn chỉnh

In [13]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model=512, n_head=8, 
                 num_encoder_layers=6, num_decoder_layers=6, d_ff=2048, dropout=0.1, max_len=5000):
        super(Transformer, self).__init__()
        
        self.src_embedding = Embeddings(src_vocab_size, d_model)
        self.trg_embedding = Embeddings(trg_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, n_head, d_ff, dropout) for _ in range(num_encoder_layers)
        ])
        
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, n_head, d_ff, dropout) for _ in range(num_decoder_layers)
        ])
        
        self.fc_out = nn.Linear(d_model, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def encode(self, src, src_mask):
        x = self.dropout(self.positional_encoding(self.src_embedding(src)))
        for layer in self.encoder_layers:
            x = layer(x, src_mask)
        return x

    def decode(self, trg, memory, src_mask, trg_mask):
        x = self.dropout(self.positional_encoding(self.trg_embedding(trg)))
        for layer in self.decoder_layers:
            x = layer(x, memory, src_mask, trg_mask)
        return x

    def forward(self, src, trg, src_mask, trg_mask):
        memory = self.encode(src, src_mask)
        output = self.decode(trg, memory, src_mask, trg_mask)
        return self.fc_out(output)

# Tạo Mask
def make_pad_mask(seq, pad_idx):
    # Mask che đi các vị trí padding: [batch, 1, 1, seq_len]
    return (seq != pad_idx).unsqueeze(1).unsqueeze(2)

def make_no_peak_mask(seq):
    # Mask tam giác để Decoder không nhìn thấy tương lai
    seq_len = seq.size(1)
    mask = torch.tril(torch.ones(seq_len, seq_len)).type(torch.uint8).to(device)
    return mask.unsqueeze(0).unsqueeze(0) # [1, 1, seq_len, seq_len]

### Phần C: Huấn luyện và Đánh giá

##### 1. Hàm Loss & Optimizer

In [14]:
# Khởi tạo mô hình
d_model = 256
n_head = 8
num_layers = 3 # Giảm số layer để train nhanh demo
d_ff = 512
dropout = 0.1

model = Transformer(len(src_vocab), len(trg_vocab), d_model, n_head, 
                    num_layers, num_layers, d_ff, dropout).to(device)

def init_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(init_weights)

optimizer = optim.Adam(model.parameters(), lr=0.0005, betas=(0.9, 0.98), eps=1e-9)
criterion = nn.CrossEntropyLoss(ignore_index=trg_vocab.stoi["<PAD>"])

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Mô hình có {count_parameters(model):,} tham số')

Mô hình có 128,558,407 tham số


##### 2. Vòng lặp huấn luyện

(Chạy 1 tiếng rồi máy tạch) 😅

In [ ]:
def train_epoch(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, (src, trg) in enumerate(iterator):
        src = src.to(device)
        trg = trg.to(device)
        
        trg_input = trg[:, :-1] # Bỏ token cuối <EOS>
        trg_label = trg[:, 1:]  # Bỏ token đầu <SOS> - đây là cái cần dự đoán
        
        # Tạo mask
        src_mask = make_pad_mask(src, src_vocab.stoi["<PAD>"])
        trg_pad_mask = make_pad_mask(trg_input, trg_vocab.stoi["<PAD>"])
        trg_no_peak_mask = make_no_peak_mask(trg_input)
        trg_mask = trg_pad_mask & trg_no_peak_mask
        
        optimizer.zero_grad()
        output = model(src, trg_input, src_mask, trg_mask)
        
        # output: [batch_size, seq_len, output_dim] -> [batch_size * seq_len, output_dim]
        # label: [batch_size * seq_len]
        output_dim = output.shape[-1]
        loss = criterion(output.contiguous().view(-1, output_dim), trg_label.contiguous().view(-1))
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

# Chạy huấn luyện
N_EPOCHS = 5
CLIP = 1

for epoch in range(N_EPOCHS):
    train_loss = train_epoch(model, dataloader, optimizer, criterion, CLIP)
    print(f'Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

### Phần D: Báo cáo kết quả (Inference & BLEU)

##### 1. Greedy Search & Beam Search

In [24]:
def beam_search_decode(model, src_sentence, src_vocab, trg_vocab, max_len=50, beam_size=3):
    model.eval()
    
    # Chuẩn bị input
    tokens = src_vocab.tokenizer(src_sentence)
    src_indexes = [src_vocab.stoi.get(token, src_vocab.stoi["<UNK>"]) for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_mask = make_pad_mask(src_tensor, src_vocab.stoi["<PAD>"])
    
    with torch.no_grad():
        encoder_output = model.encode(src_tensor, src_mask)
    
    # Beam Search: list chứa (score, sequence_tensor)
    sequences = [[0.0, [trg_vocab.stoi["<SOS>"]]]]
    
    for _ in range(max_len):
        all_candidates = []
        
        for score, seq in sequences:
            # Nếu câu đã kết thúc bằng <EOS>, giữ nguyên
            if seq[-1] == trg_vocab.stoi["<EOS>"]:
                all_candidates.append([score, seq])
                continue
            
            trg_tensor = torch.LongTensor(seq).unsqueeze(0).to(device)
            trg_mask = make_no_peak_mask(trg_tensor) & make_pad_mask(trg_tensor, trg_vocab.stoi["<PAD>"])
            
            output = model.decode(trg_tensor, encoder_output, src_mask, trg_mask)
            prob = output[:, -1, :] # Lấy từ cuối cùng
            log_prob = torch.nn.functional.log_softmax(prob, dim=-1)
            
            # Lấy top k candidate từ từ điển
            top_k_log_prob, top_k_idx = log_prob.topk(beam_size)
            
            for i in range(beam_size):
                current_log_prob = top_k_log_prob[0][i].item()
                candidate_idx = top_k_idx[0][i].item()
                
                # Điểm mới = điểm cũ + log_prob của từ mới (vì log(p1*p2) = log(p1) + log(p2))
                new_score = score + current_log_prob
                new_seq = seq + [candidate_idx]
                all_candidates.append([new_score, new_seq])
        
        # Sắp xếp theo score cao nhất và chọn ra beam_size câu tốt nhất
        sequences = sorted(all_candidates, key=lambda x: x[0], reverse=True)[:beam_size]
        
        # Nếu tất cả các beam đều đã gặp <EOS> thì dừng
        if all([seq[-1] == trg_vocab.stoi["<EOS>"] for _, seq in sequences]):
            break
            
    # Lấy câu có điểm cao nhất
    best_seq = sequences[0][1]
    
    # Convert ID sang từ
    trg_tokens = [trg_vocab.itos[idx] for idx in best_seq if idx not in [trg_vocab.stoi["<SOS>"], trg_vocab.stoi["<EOS>"]]]
    return " ".join(trg_tokens)

# Test thử
test_sentence = "Tôi đi học" # Thay bằng câu có trong vocab của bạn
translation = beam_search_decode(model, test_sentence, src_vocab, trg_vocab, beam_size=3)
print(f"Câu nguồn: {test_sentence}")
print(f"Dịch máy: {translation}")

Câu nguồn: Tôi đi học
Dịch máy: diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic diagnostic


##### 2. Đánh giá BLEU Score

In [18]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [23]:
# 1. Đọc dữ liệu từ file public_test
try:
    # Đọc file tiếng Việt (nguồn)
    with open('public_test.vi.txt', 'r', encoding='utf-8') as f:
        public_test_src = f.readlines()
    
    # Đọc file tiếng Anh (đích)
    with open('public_test.en.txt', 'r', encoding='utf-8') as f:
        public_test_trg = f.readlines()

    print(f"Đã tải {len(public_test_src)} câu kiểm thử.")

    # 2. Hàm tính BLEU Score (Sử dụng NLTK)
    import nltk
    from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
    
    # Download dữ liệu tokenize của nltk nếu chưa có (chạy 1 lần)
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')

    def evaluate_test_set(model, src_lines, trg_lines, src_vocab, trg_vocab, limit=None):
        model.eval()
        total_bleu = 0
        count = 0
        
        # Nếu không giới hạn thì chạy hết, ngược lại chỉ chạy số lượng 'limit' câu để test nhanh
        num_sentences = len(src_lines)
        if limit is not None:
            num_sentences = min(num_sentences, limit)
            
        print(f"Đang đánh giá trên {num_sentences} câu...")
        
        # Dùng smoothing để tránh điểm 0 khi câu quá ngắn hoặc không khớp n-gram cao
        smoothie = SmoothingFunction().method1

        for i in range(num_sentences):
            src_text = src_lines[i]
            trg_text = trg_lines[i]
            
            # Dịch câu nguồn
            pred_text = beam_search_decode(model, src_text, src_vocab, trg_vocab, beam_size=3)
            
            # Chuẩn bị dữ liệu để tính BLEU
            # Tách từ cho câu tham chiếu (Reference)
            reference = [trg_vocab.tokenizer(trg_text)] 
            # Tách từ cho câu dự đoán (Candidate)
            candidate = trg_vocab.tokenizer(pred_text)
            
            # Tính điểm cho từng câu
            score = sentence_bleu(reference, candidate, smoothing_function=smoothie)
            total_bleu += score
            count += 1
            
            # In thử vài câu đầu để xem kết quả dịch
            if i < 5:
                print(f"\n--- Câu {i+1} ---")
                print(f"Nguồn: {src_text.strip()}")
                print(f"Đích (Chuẩn): {trg_text.strip()}")
                print(f"Máy dịch: {pred_text}")
                print(f"Điểm BLEU: {score:.4f}")

        avg_bleu = total_bleu / count
        return avg_bleu

    # 3. Chạy đánh giá
    # Lưu ý: limit=100 nghĩa là chỉ test trên 100 câu đầu cho nhanh. 
    # Muốn test hết thì bỏ tham số limit đi: evaluate_test_set(..., limit=None)
    bleu_score = evaluate_test_set(model, public_test_src, public_test_trg, src_vocab, trg_vocab, limit=50)

    print(f"\n==========================================")
    print(f"KẾT QUẢ ĐÁNH GIÁ TRÊN PUBLIC TEST")
    print(f"Trung bình BLEU Score: {bleu_score * 100:.2f}")
    print(f"==========================================")

except FileNotFoundError:
    print("Lỗi: Không tìm thấy file 'public_test.vi.txt' hoặc 'public_test.en.txt'. Vui lòng kiểm tra lại thư mục.")
except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

Đã tải 3000 câu kiểm thử.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Đang đánh giá trên 50 câu...
Có lỗi xảy ra: 'int' object is not subscriptable
